### 1. Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pyodbc
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

### 2. Merge .pkl's

In [2]:
# Zuigelingen
zuigelingen_observaties = pd.read_pickle('4_Data\Pickles\zuigelingen_observaties.pkl')
zuigelingen_relaties = pd.read_pickle('4_Data\Pickles\zuigelingen_relaties.pkl')
zuigelingen_toestemmingen = pd.read_pickle('4_Data\Pickles\zuigelingen_toestemmingen.pkl')
zuigelingen_cbs_pc4 = pd.read_pickle('4_Data\Pickles\zuigelingen_cbs4.pkl')
zuigelingen_cbs_buurt = pd.read_pickle('4_Data\Pickles\zuigelingen_buurt.pkl')
zuigelingen_coronit = pd.read_pickle('4_Data\Pickles\zuigelingen_coronit.pkl')

In [3]:
# Merge zuigelingen_observaties, zuigelingen_PC6 and zuigelingen_relaties on 'Clientnummer'
zuigelingen = (zuigelingen_observaties
                 .merge(zuigelingen_toestemmingen, on='Clientnummer', how='left')
                 .merge(zuigelingen_cbs_pc4, on='Clientnummer', how='left')
                 .merge(zuigelingen_cbs_buurt, on='Clientnummer', how='left')
                 .merge(zuigelingen_relaties, on='Clientnummer', how='left'))

In [4]:
len(zuigelingen)

5384

#### 2.1 Parent 1

In [5]:
# Change the dtype of zuigelingen['Huisnummer_BRP'] to int64
zuigelingen['Huisnummer_BRP'] = zuigelingen['Huisnummer_BRP'].astype('int64')

##### 2.1.1 Merge CoronIT on all four characteristics

In [6]:
# Merge zuigelingen with df_coronit
zuigelingen_coronit = zuigelingen_coronit.rename(columns={'postcode': 'Postcode6_BRP',
                                                              'huisnummer': 'Huisnummer_BRP',
                                                              'geboortedatum_ym': 'Geboortejaarmaand_relatie_ouder1',
                                                              'geslacht': 'Geslacht_relatie_ouder1',
                                                              'gevaccineerd': 'Gevaccineerd_ouder1'})
zuigelingen = zuigelingen.merge(zuigelingen_coronit, on=['Postcode6_BRP', 'Huisnummer_BRP', 'Geslacht_relatie_ouder1', 'Geboortejaarmaand_relatie_ouder1'], how='left')

In [7]:
# Print the missingness percentage of Gevaccineerd_ouder1
zuigelingen['Gevaccineerd_ouder1'].isnull().mean()

0.24479940564635957

##### 2.1.2 Merge CoronIT after discarding sex (only for NA's after 2.1)

In [8]:
# Get the subset of zuigelingen where Gevaccineerd_ouder1 is missing
missing_gevaccineerd = zuigelingen[zuigelingen['Gevaccineerd_ouder1'].isnull()]

# From zuigelingen_coronit, remove columns 'Geslacht_relatie_ouder1'
zuigelingen_coronit = zuigelingen_coronit.drop(columns='Geslacht_relatie_ouder1')

# Merge again, but without Geslacht_relatie_ouder1
updated_missing = missing_gevaccineerd.merge(zuigelingen_coronit, on=['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder1'], how='left')

# From updated_missing, remove columns 'Gevaccineerd_ouder1_x'
updated_missing = updated_missing.drop(columns='Gevaccineerd_ouder1_x')

# Concatenate the updated subset back with the non-missing part
non_missing = zuigelingen[~zuigelingen['Gevaccineerd_ouder1'].isnull()]
zuigelingen = pd.concat([non_missing, updated_missing], ignore_index=True)

# Use combine_first to combine Gevaccineerd_ouder1 and Gevaccineerd_ouder1_y
zuigelingen['Gevaccineerd_ouder1'] = zuigelingen['Gevaccineerd_ouder1'].combine_first(zuigelingen['Gevaccineerd_ouder1_y'])

# Drop Gevaccineerd_ouder1_y
zuigelingen = zuigelingen.drop('Gevaccineerd_ouder1_y', axis=1)

In [9]:
# Print the missingness percentage of Gevaccineerd_ouder1
zuigelingen['Gevaccineerd_ouder1'].isnull().mean()

0.10289747399702823

In [10]:
zuigelingen['Gevaccineerd_ouder1'].value_counts(normalize=True)

Gevaccineerd_ouder1
True     0.693996
False    0.306004
Name: proportion, dtype: float64

#### 2.2 Parent 2

In [11]:
zuigelingen_coronit = pd.read_pickle('4_Data\Pickles\zuigelingen_coronit.pkl')

##### 2.2.1 Merge CoronIT on all four characteristics

In [12]:
# Merge zuigelingen with df_coronit
zuigelingen_coronit = zuigelingen_coronit.rename(columns={'postcode': 'Postcode6_BRP',
                                                              'huisnummer': 'Huisnummer_BRP',
                                                              'geboortedatum_ym': 'Geboortejaarmaand_relatie_ouder2',
                                                              'geslacht': 'Geslacht_relatie_ouder2',
                                                              'gevaccineerd': 'Gevaccineerd_ouder2'})
zuigelingen = zuigelingen.merge(zuigelingen_coronit, on=['Postcode6_BRP', 'Huisnummer_BRP', 'Geslacht_relatie_ouder2', 'Geboortejaarmaand_relatie_ouder2'], how='left')

In [13]:
# Print the missingness percentage of Gevaccineerd_ouder1
zuigelingen['Gevaccineerd_ouder2'].isnull().mean()

0.2635586924219911

##### 2.2.2 Merge CoronIT after discarding sex (only for NA's after 2.1)

In [14]:
# Get the subset of zuigelingen where Gevaccineerd_ouder1 is missing
missing_gevaccineerd = zuigelingen[zuigelingen['Gevaccineerd_ouder2'].isnull()]

# From zuigelingen_coronit, remove columns 'Geslacht_relatie_ouder1'
zuigelingen_coronit = zuigelingen_coronit.drop(columns='Geslacht_relatie_ouder2')

# Merge again, but without Geslacht_relatie_ouder1
updated_missing = missing_gevaccineerd.merge(zuigelingen_coronit, on=['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder2'], how='left')

# From updated_missing, remove columns 'Gevaccineerd_ouder1_x'
updated_missing = updated_missing.drop(columns='Gevaccineerd_ouder2_x')

# Concatenate the updated subset back with the non-missing part
non_missing = zuigelingen[~zuigelingen['Gevaccineerd_ouder2'].isnull()]
zuigelingen = pd.concat([non_missing, updated_missing], ignore_index=True)

# Use combine_first to combine Gevaccineerd_ouder1 and Gevaccineerd_ouder1_y
zuigelingen['Gevaccineerd_ouder2'] = zuigelingen['Gevaccineerd_ouder2'].combine_first(zuigelingen['Gevaccineerd_ouder2_y'])

# Drop Gevaccineerd_ouder1_y
zuigelingen = zuigelingen.drop('Gevaccineerd_ouder2_y', axis=1)

In [15]:
# Print the missingness percentage of Gevaccineerd_ouder1
zuigelingen['Gevaccineerd_ouder2'].isnull().mean()

0.11367013372956909

In [16]:
zuigelingen['Gevaccineerd_ouder2'].value_counts(normalize=True)

Gevaccineerd_ouder2
True     0.696982
False    0.303018
Name: proportion, dtype: float64

### 3. Apply transformations

#### 3.1 Parent matching

In [17]:
# If both Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are not null and not equal, switch the columns
mask = ((zuigelingen['Geboortejaarmaand_relatie_ouder1'].notnull()) &
        (zuigelingen['GeboortejaarmaandBiologischeMoeder'].notnull()) &
        (zuigelingen['Geboortejaarmaand_relatie_ouder1'] != zuigelingen['GeboortejaarmaandBiologischeMoeder']) &
        (zuigelingen['Geboortejaarmaand_relatie_ouder2'] == zuigelingen['GeboortejaarmaandBiologischeMoeder']))

# Switch
zuigelingen.loc[mask, ['Geboortejaarmaand_relatie_ouder1',
                       'Geboorteland_relatie_ouder1',
                       'Geslacht_relatie_ouder1',
                       'Geboortejaarmaand_relatie_ouder2',
                       'Geboorteland_relatie_ouder2',
                       'Geslacht_relatie_ouder2']] = zuigelingen.loc[mask, ['Geboortejaarmaand_relatie_ouder2',
                                                                            'Geboorteland_relatie_ouder2',
                                                                            'Geslacht_relatie_ouder2',
                                                                            'Geboortejaarmaand_relatie_ouder1',
                                                                            'Geboorteland_relatie_ouder1',
                                                                            'Geslacht_relatie_ouder1']].values

In [18]:
print(f'The amount of relations in the df_observaties_relaties is: {len(zuigelingen)}, the amount of unique client ID\'s is: {len(zuigelingen['Clientnummer'].unique())}')

The amount of relations in the df_observaties_relaties is: 5384, the amount of unique client ID's is: 5384


In [19]:
# Set all 'onbekend' values in zuigelingen['Geboortejaarmaand_relatie_ouder1'] to np.nan
zuigelingen = zuigelingen.replace('onbekend', np.nan)

# Print matches of parents
print(f'The amount of rows where both Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are empty: {len(zuigelingen[(zuigelingen['Geboortejaarmaand_relatie_ouder1'].isnull()) & (zuigelingen['GeboortejaarmaandBiologischeMoeder'].isnull())])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 is empty and GeboortejaarmaandBiologischeMoeder is not: {len(zuigelingen[(zuigelingen['Geboortejaarmaand_relatie_ouder1'].isnull()) & (zuigelingen['GeboortejaarmaandBiologischeMoeder'].notnull())])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is empty: {len(zuigelingen[(zuigelingen['Geboortejaarmaand_relatie_ouder1'].notnull()) & (zuigelingen['GeboortejaarmaandBiologischeMoeder'].isnull())])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is not empty and they are not equal: {len(zuigelingen[(zuigelingen['Geboortejaarmaand_relatie_ouder1'].notnull()) & (zuigelingen['GeboortejaarmaandBiologischeMoeder'].notnull()) & (zuigelingen['Geboortejaarmaand_relatie_ouder1'] != zuigelingen['GeboortejaarmaandBiologischeMoeder'])])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are equal: {len(zuigelingen[(zuigelingen['Geboortejaarmaand_relatie_ouder1'].notnull()) & (zuigelingen['GeboortejaarmaandBiologischeMoeder'].notnull()) & (zuigelingen['Geboortejaarmaand_relatie_ouder1'] == zuigelingen['GeboortejaarmaandBiologischeMoeder'])])}')

The amount of rows where both Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are empty: 64
The amount of rows where Geboortejaarmaand_relatie_ouder1 is empty and GeboortejaarmaandBiologischeMoeder is not: 10
The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is empty: 670
The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is not empty and they are not equal: 4640
The amount of rows where Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are equal: 0


In [20]:
# Merge Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder
zuigelingen['Geboortejaarmaand_relatie_ouder1'] = zuigelingen['Geboortejaarmaand_relatie_ouder1'].combine_first(zuigelingen['GeboortejaarmaandBiologischeMoeder'])

#### 3.2 Migrants (CBS Indeling)

In [21]:
# Create substitution dictionary
migrants_cat = {'Syrië': 'Syrië',
                    'Nederland' : 'Nederland',
                                                                   'Maleisië': 'Overig Azië',
                                                                   'België': 'Europa (excl. NL)',
                                                                   'Polen': 'Europa (excl. NL)',
                                                                   'Verenigde Staten van Amerika': 'Overig Amerika en Oceanië',
                                                                   'Spanje': 'Europa (excl. NL)',
                                                                   'Portugal': 'Europa (excl. NL)',
                                                                   'Groot-Brittannië': 'Europa (excl. NL)',
                                                                   'Afghanistan': 'Overig Azië',
                                                                   'Oekraïne': 'Europa (excl. NL)',
                                                                   'Zuid-Afrika': 'Overig Afrika',
                                                                   'Denemarken': 'Europa (excl. NL)',
                                                                   'Canada': 'Overig Amerika en Oceanië',
                                                                   'Roemenië': 'Europa (excl. NL)',
                                                                   'Bondsrepubliek Duitsland': 'Europa (excl. NL)',
                                                                   'Ecuador': 'Overig Amerika en Oceanië',
                                                                   'Turkije': 'Turkije',
                                                                   'Jordanië': 'Overig Azië',
                                                                   'Israël': 'Overig Azië',
                                                                   'Libanon': 'Overig Azië',
                                                                   'Cyprus': 'Overig Azië', # Cyprus is geografisch gezien in Azië, maar politiek gezien in Europa
                                                                   'Brazilië': 'Overig Amerika en Oceanië',
                                                                   'Thailand': 'Overig Azië',
                                                                   'Singapore': 'Overig Azië',
                                                                   'Bulgarije': 'Europa (excl. NL)',
                                                                   'Zuid-Korea': 'Overig Azië',
                                                                   'Griekenland': 'Europa (excl. NL)',
                                                                   'China': 'Overig Azië',
                                                                   'Suriname': 'Suriname',
                                                                   'Chili': 'Overig Amerika en Oceanië',
                                                                   'Indonesië': 'Indonesië',
                                                                   'Marokko': 'Marokko',
                                                                   'Finland': 'Europa (excl. NL)',
                                                                   'Onbekend': np.nan,
                                                                   'Filipijnen': 'Overig Azië',
                                                                   'Armenië': 'Overig Azië',
                                                                   'Frankrijk': 'Europa (excl. NL)',
                                                                   'Iran': 'Overig Azië',
                                                                   'Hongarije': 'Europa (excl. NL)',
                                                                   'Dominicaanse Republiek': 'Overig Amerika en Oceanië',
                                                                   'India': 'Overig Azië',
                                                                   'Joegoslavië': 'Europa (excl. NL)',
                                                                   'Irak': 'Overig Azië',
                                                                   'Oezbekistan': 'Overig Azië',
                                                                   'Sri Lanka': 'Overig Azië',
                                                                   'Nederlandse Antillen': 'Nederlandse Cariben',
                                                                   'Sovjet-Unie': 'Europa (excl. NL)',
                                                                   'Litouwen': 'Europa (excl. NL)',
                                                                   'Ethiopië': 'Overig Afrika',
                                                                   'Korea': 'Overig Azië',
                                                                   'Soedan': 'Overig Afrika',
                                                                   'Somalië': 'Overig Afrika',
                                                                   'Saoedi-Arabië': 'Overig Azië',
                                                                   'Verenigde Arabische Emiraten': 'Overig Azië',
                                                                   'Australië': 'Overig Amerika en Oceanië',
                                                                   'Slowakije': 'Europa (excl. NL)',
                                                                   'Vietnam': 'Overig Azië',
                                                                   'Estland': 'Europa (excl. NL)',
                                                                   'Zaïre': 'Overig Afrika',
                                                                   'Zambia': 'Overig Afrika',
                                                                   'Noorwegen': 'Europa (excl. NL)',
                                                                   'Panama': 'Overig Amerika en Oceanië',
                                                                   'Peru': 'Overig Amerika en Oceanië',
                                                                   'Algerije': 'Overig Afrika',
                                                                   'Guatemala': 'Overig Amerika en Oceanië',
                                                                   'Zwitserland': 'Europa (excl. NL)',
                                                                   'Ghana': 'Overig Afrika',
                                                                   'Italië': 'Europa (excl. NL)',
                                                                   'Duitse Democratische Republiek': 'Europa (excl. NL)',
                                                                   'Eritrea': 'Overig Afrika',
                                                                   'Nigeria': 'Overig Afrika',
                                                                   'Colombia': 'Overig Amerika en Oceanië',
                                                                   'Macedonië': 'Europa (excl. NL)',
                                                                   'Egypte': 'Overig Afrika',
                                                                   'Pakistan': 'Overig Azië',
                                                                   'Nieuw-Zeeland': 'Overig Amerika en Oceanië',
                                                                   'Kameroen': 'Overig Afrika',
                                                                   'Guinee': 'Overig Afrika',
                                                                   'Sierra Leone': 'Overig Afrika',
                                                                   'Libië': 'Overig Afrika',
                                                                   'Aruba': 'Nederlandse Cariben',
                                                                   'Tsjecho-Slowakije': 'Europa (excl. NL)',
                                                                   'Azerbeidzjan': 'Overig Azië',
                                                                   'Uganda': 'Overig Afrika',
                                                                   'Democratische Republiek Congo': 'Overig Afrika',
                                                                   'nan': np.nan,
                                                                   'Burundi': 'Overig Afrika',
                                                                   'Gazastrook en Westelijke Jordaanoever': 'Overig Azië',
                                                                   'Gambia': 'Overig Afrika',
                                                                   'Angola': 'Overig Afrika',
                                                                   'Japan': 'Overig Azië',
                                                                   'Venezuela': 'Overig Amerika en Oceanië',
                                                                   'Kenya': 'Overig Afrika',
                                                                   'Federale Republiek Joegoslavië': 'Europa (excl. NL)'}

# Map all unique values to an overarching category
zuigelingen['Geboorteland_cat'] = zuigelingen['Geboorteland'].map(migrants_cat)
zuigelingen['Geboorteland_relatie_ouder1_cat'] = zuigelingen['Geboorteland_relatie_ouder1'].map(migrants_cat)
zuigelingen['Geboorteland_relatie_ouder2_cat'] = zuigelingen['Geboorteland_relatie_ouder2'].map(migrants_cat)

In [22]:
# Create substitution dictionary
migrants_cat3 = {'Syrië': 'Niet-Westers',
                    'Nederland': 'Nederlands',
                    'Marokko': 'Niet-Westers',
                    'Europa (excl. NL)': 'Westers',
                    'Turkije': 'Niet-Westers',
                    'Nederlandse Cariben': 'Niet-Westers',
                    'Overig Azië': 'Niet-Westers',
                    'Overig Amerika en Oceanië': 'Westers',
                    'Suriname': 'Niet-Westers',
                    'Overig Afrika': 'Niet-Westers',
                    'Indonesië': 'Niet-Westers'}

# Map all unique values to an overarching category
zuigelingen['Geboorteland_cat3'] = zuigelingen['Geboorteland_cat'].map(migrants_cat3)
zuigelingen['Geboorteland_relatie_ouder1_cat3'] = zuigelingen['Geboorteland_relatie_ouder1_cat'].map(migrants_cat3)
zuigelingen['Geboorteland_relatie_ouder2_cat3'] = zuigelingen['Geboorteland_relatie_ouder2_cat'].map(migrants_cat3)

In [23]:
# Create new column 'CBS_indeling'
zuigelingen['CBS_indeling'] = zuigelingen.apply(
    lambda x: (
        'Nederlands' if x['Geboorteland_cat3'] == 'Nederlands' and x['Geboorteland_relatie_ouder1_cat3'] == 'Nederlands' and x['Geboorteland_relatie_ouder2_cat3'] == 'Nederlands'
        else (
            'Kindmigrant' if x['Geboorteland_cat3'] == 'Nederlands' and (
                x['Geboorteland_relatie_ouder1_cat3'] in ['Westers', 'Niet-Westers'] or x['Geboorteland_relatie_ouder2_cat3'] in ['Westers', 'Niet-Westers']
            )
            else (
                'Migrant' if x['Geboorteland_cat3'] in ['Westers', 'Niet-Westers']
                else np.nan
            )
        )
    ), axis=1
)

In [24]:
#Count occurrences of CBS_indeling including missing values
zuigelingen['CBS_indeling'].value_counts(dropna=False)

CBS_indeling
Nederlands     4412
Kindmigrant     811
NaN             149
Migrant          12
Name: count, dtype: int64

### 4. Create CoronIT unique keys

In [25]:
# Print the percentage of missing values per column
print(f'The percentage of missing values per column is:')
for col in zuigelingen.columns:
    print(f'{col:<100}: {round(zuigelingen[col].isnull().mean(), 2)}')

The percentage of missing values per column is:
Clientnummer                                                                                        : 0.0
B.M.R.                                                                                              : 0.0
DKTP                                                                                                : 0.0
DTP                                                                                                 : 0.0
Hib                                                                                                 : 0.0
Pneu                                                                                                : 0.0
Excluded                                                                                            : 0.0
Hepatitis_B                                                                                         : 0.0
MenACWY/MenC                                                                                        : 0.

In [26]:
# Remove twins from zuigelingen
print(f'{len(zuigelingen[zuigelingen['Tweeling'] == True])} individual twins')
print(f'{len(zuigelingen)} individuals in total')
zuigelingen_without_twins = zuigelingen[zuigelingen['Tweeling'] == False][['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder1', 'Geslacht_relatie_ouder1', 'Geboortejaarmaand_relatie_ouder2', 'Geslacht_relatie_ouder2']]
zuigelingen_twins = zuigelingen[zuigelingen['Tweeling'] == True][['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder1', 'Geslacht_relatie_ouder1', 'Geboortejaarmaand_relatie_ouder2', 'Geslacht_relatie_ouder2']]
zuigelingen_twins = zuigelingen_twins.drop_duplicates()
zuigelingen_without_twins = pd.concat([zuigelingen_without_twins, zuigelingen_twins])
print(f'{len(zuigelingen_without_twins)} individuals after removing twins')

100 individual twins
5384 individuals in total
5335 individuals after removing twins


In [27]:
# Get combinations representing first parents
parent_1 = zuigelingen_without_twins[['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder1', 'Geslacht_relatie_ouder1']]
parent_1.columns = ['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder', 'Geslacht_relatie_ouder']

# Check
parent_1 = parent_1.dropna(subset=['Geboortejaarmaand_relatie_ouder'])
print(f'{len(parent_1)} first parents after deleting rows with missings')

5271 first parents after deleting rows with missings


In [28]:
# Get combinations representing second parents
parent_2 = zuigelingen_without_twins[['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder2', 'Geslacht_relatie_ouder2']]
parent_2.columns = ['Postcode6_BRP', 'Huisnummer_BRP', 'Geboortejaarmaand_relatie_ouder', 'Geslacht_relatie_ouder']

# Check
parent_2 = parent_2.dropna(subset=['Geboortejaarmaand_relatie_ouder'])
print(f'{len(parent_2)} second parents after deleting rows with missings')

5193 second parents after deleting rows with missings


In [29]:
# Stack the two dataframes, reset indices
parents = pd.concat([parent_1, parent_2]).reset_index(drop=True)
print(f'{len(parents)} parents in total')

# Show duplicate rows
print('-'*20)
print(f'{len(parents[parents.duplicated()])} duplicate parents')
parents = parents.drop_duplicates()
print(f'{len(parents)} parents after dropping duplicates')

10464 parents in total
--------------------
260 duplicate parents
10204 parents after dropping duplicates


In [30]:
# Print the percentages of missing values per column in parents
print(f'The percentage of missing values per column in parents is:')
for col in parents.columns:
    print(f'{col:<70}: {round(parents[col].isnull().mean(), 2)}')

The percentage of missing values per column in parents is:
Postcode6_BRP                                                         : 0.0
Huisnummer_BRP                                                        : 0.0
Geboortejaarmaand_relatie_ouder                                       : 0.0
Geslacht_relatie_ouder                                                : 0.16


In [31]:
# Save parents as .csv
parents.to_csv('4_Data\CoronIT\keys\Ouders_zuigelingen.csv', index=False)

### 5. Save .pkl's

In [32]:
# Save df_contactmomenten_og as a pickle
zuigelingen.to_pickle('4_Data\Pickles\zuigelingen.pkl')